In [1]:
# 連結 Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
cd "/content/drive/My Drive/AI homework"

/content/drive/My Drive/AI homework


In [0]:
import cv2
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt

In [0]:
#!unzip images.zip

In [0]:
def openfile(file):
    f = open(file,"r", encoding='utf-8')
    lines = f.readlines()

    image_paths=[]  
    image_classes=[]
    for line in lines:
        curLine=line.strip().split(" ")
        image_paths.append(curLine[0]) 
        image_classes.append(curLine[-1])

    time=1
    allt=len(image_paths)
    ims = []
    print("loading images")
    for image_path in image_paths:
        img  = cv2.imread(image_path)
        img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        img=cv2.resize(img,(64,64),interpolation=cv2.INTER_CUBIC)
        ims.append(img)
        print(str(time)+"/"+str(allt) +"  " + image_path, end="\r")
        time = time +1
    print("\nDone!")
    return(ims,image_paths,image_classes)

In [0]:
class Matcher(object):

    def __init__(self, pickled_db_path="features.pck"):
        with open(pickled_db_path) as fp:
            self.data = cPickle.load(fp)
        self.names = []
        self.matrix = []
        for k, v in self.data.iteritems():
            self.names.append(k)
            self.matrix.append(v)
        self.matrix = np.array(self.matrix)
        self.names = np.array(self.names)

    def cos_cdist(self, vector):
        # getting cosine distance between search image and images database
        v = vector.reshape(1, -1)
        return scipy.spatial.distance.cdist(self.matrix, v, 'cosine').reshape(-1)

    def match(self, image_path, topn=5):
        features = extract_features(image_path)
        img_distances = self.cos_cdist(features)
        # getting top 5 records
        nearest_ids = np.argsort(img_distances)[:topn].tolist()
        nearest_img_paths = self.names[nearest_ids].tolist()

        return nearest_img_paths, img_distances[nearest_ids].tolist()

In [0]:
# Feature extractor
def extract_features(image_path, vector_size=32):
    image = imread(image_path, mode="RGB")
    try:
        # Using KAZE, cause SIFT, ORB and other was moved to additional module
        # which is adding addtional pain during install
        alg = cv2.SIFT_create()
        # Dinding image keypoints
        kps = alg.detect(image)
        # Getting first 32 of them. 
        # Number of keypoints is varies depend on image size and color pallet
        # Sorting them based on keypoint response value(bigger is better)
        kps = sorted(kps, key=lambda x: -x.response)[:vector_size]
        # computing descriptors vector
        kps, dsc = alg.compute(image, kps)
        # Flatten all of them in one big vector - our feature vector
        dsc = dsc.flatten()
        # Making descriptor of same size
        # Descriptor vector size is 64
        needed_size = (vector_size * 64)
        if dsc.size < needed_size:
            # if we have less the 32 descriptors then just adding zeros at the
            # end of our feature vector
            dsc = np.concatenate([dsc, np.zeros(needed_size - dsc.size)])
    except cv2.error as e:
        print('Error: ', e)
        return None

    return dsc


def batch_extractor(images_path, pickled_db_path="features.pck"):
    files = [os.path.join(images_path, p) for p in sorted(os.listdir(images_path))]

    result = {}
    for f in files:
        print ('Extracting features from image %s' % f)
        name = f.split('/')[-1].lower()
        result[name] = extract_features(f)
    
    # saving all our feature vectors in pickled file
    with open(pickled_db_path, 'w') as fp:
        cPickle.dump(result, fp)

In [0]:
class Matcher(object):

    def __init__(self, pickled_db_path="features.pck"):
        with open(pickled_db_path) as fp:
            self.data = cPickle.load(fp)
        self.names = []
        self.matrix = []
        for k, v in self.data.iteritems():
            self.names.append(k)
            self.matrix.append(v)
        self.matrix = np.array(self.matrix)
        self.names = np.array(self.names)

    def cos_cdist(self, vector):
        # getting cosine distance between search image and images database
        v = vector.reshape(1, -1)
        return scipy.spatial.distance.cdist(self.matrix, v, 'cosine').reshape(-1)

    def match(self, image_path, topn=5):
        features = extract_features(image_path)
        img_distances = self.cos_cdist(features)
        # getting top 5 records
        nearest_ids = np.argsort(img_distances)[:topn].tolist()
        nearest_img_paths = self.names[nearest_ids].tolist()

        return nearest_img_paths, img_distances[nearest_ids].tolist()

In [0]:
#train
ims=[]
image_paths=[]
image_classes=[]
ims,image_paths,image_classes=openfile("train.txt")
sift_features,sift_image_classes=siftfeature(image_paths,image_classes,ims)

loading images


In [0]:
#val
imsval=[]
image_pathsval=[]
image_classesval=[]
imsval,image_pathsval,image_classesval=openfile("val.txt")
val_sift_features,sift_image_classesval=siftfeature(image_pathsval,image_classesval,imsval)
siftclf,siftpredictions=siftSVC(sift_features,sift_image_classes,val_sift_features)

In [0]:
ls

In [0]:
#test
imstest=[]
image_pathstest=[]
image_classestest=[]
imstest,image_pathstest,image_classestest=openfile("test.txt")
test_sift_features,sift_image_classestest=siftfeature(image_pathstest,image_classestest,imstest)

In [0]:
siftpredictions=predicts(siftclf,test_sift_features)

In [0]:
acc_sift=round((accuracy_score(sift_image_classestest, siftpredictions)),3)
aa={"sift":acc_sift}
id=["acc"]
ans=pd.DataFrame(aa,index=id)

In [0]:
print(ans)

In [0]:
ls